# Student Loan Risk with Deep Learning

In [1]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [2]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [3]:
# Review the data types associated with the columns
loans_df.dtypes

payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object

In [4]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

credit_ranking
1    855
0    744
Name: count, dtype: int64

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [7]:
# Define the target set y using the credit_ranking column
y = loans_df['credit_ranking']

# Display a sample of y
y[:5]

0    0
1    0
2    0
3    1
4    0
Name: credit_ranking, dtype: int64

In [9]:
# Define features set X by selecting all columns but credit_ranking
X = loans_df.drop(columns="credit_ranking", axis=1)

# Review the features DataFrame
X.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4


### Step 3: Split the features and target sets into training and testing datasets.


In [10]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y)

### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [13]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_train_scaled = scaler.fit_transform(X_train)

# Fit the scaler to the features testing dataset
X_test_scaled = scaler.transform(X_test)

X_train_scaled[:5]

array([[-0.92276993,  0.08150857, -0.73193796, -0.383679  , -0.24626939,
        -0.09276616, -0.3237435 ,  0.28907638,  1.91836617, -0.09719866,
        -0.58516496],
       [ 0.9511388 , -0.9859515 ,  1.00576997, -0.383679  , -0.57167935,
        -0.9590921 , -1.07664997, -0.47557345, -0.52010608, -0.79609635,
         1.47863199],
       [-1.32026572, -0.14321986, -1.3963557 , -0.78542367, -0.65845534,
        -0.09276616, -0.62490609, -1.1980357 ,  0.31410811, -1.20378667,
        -0.86659182],
       [ 0.44007279, -1.32304416,  0.29024318, -0.35020028, -0.52829136,
        -0.28528303, -0.59478983, -0.8394413 , -0.32759512,  1.06763083,
         1.19720513],
       [ 0.4968579 , -0.53649463,  1.10798809, -0.11584922, -0.02932942,
         0.67730134,  2.0855572 ,  0.4472798 , -0.52010608, -0.09719866,
        -0.86659182]])

---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [18]:
# Define the the number of inputs (features) to the model
input_nodes = X_train.shape[1]

# Review the number of features
input_nodes

11

In [19]:
# Define the number of hidden nodes for the first hidden layer
hl1_node_count = 3

# Define the number of hidden nodes for the second hidden layer
hl2_node_count = 3

# Define the number of neurons in the output layer
ol_node_count = 1

In [22]:
# Create the Sequential model instance
nn_model = tf.keras.models.Sequential()

# Add the first hidden layer (includes input layer)
nn_model.add(tf.keras.layers.Dense(units=6, activation="relu", input_dim=input_nodes))

# Add the second hidden layer
nn_model.add(tf.keras.layers.Dense(units=3, activation="relu"))

# Add the output layer to the model specifying the number of output neurons and activation function
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

In [23]:
# Display the Sequential model summary
nn_model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 6)              │            72 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 3)              │            21 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │             4 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 97 (388.00 B)

 Trainable params: 97 (388.00 B)

 Non-trainable params: 0 (0.00 B)

### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [25]:
# Compile the Sequential model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [26]:
# Fit the model using 50 epochs and the training data
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.4715 - loss: 0.7485
Epoch 2/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4781 - loss: 0.7357 
Epoch 3/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5358 - loss: 0.7012 
Epoch 4/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5770 - loss: 0.6825 
Epoch 5/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5991 - loss: 0.6612 
Epoch 6/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5809 - loss: 0.6581
Epoch 7/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6181 - loss: 0.6323 
Epoch 8/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6437 - loss: 0.6061
Epoch 9/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6547 - loss: 0.5924 
Epoch 10/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6886 - loss: 0.5665 
Epoch 11/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6793 - loss: 0.5786
Epoch 12/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.71

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [27]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

13/13 - 0s - 14ms/step - accuracy: 0.7650 - loss: 0.4787
Loss: 0.47865772247314453, Accuracy: 0.7649999856948853


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [28]:
# Set the model's file path
path = "student_loans.keras"

# Export your model to a keras file
nn_model.save(path)

---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [29]:
# Set the model's file path


# Load the model to a new object
loaded_nn_model = tf.keras.models.load_model(path)


### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [33]:
# Make predictions with the test data
y_test_pred = loaded_nn_model.predict(X_test)

# Display a sample of the predictions
y_test_pred[:5]

13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 


array([[4.5287519e-04],
       [3.9313434e-07],
       [2.8082013e-07],
       [9.9942207e-01],
       [7.5092197e-17]], dtype=float32)

In [36]:
# Save the predictions to a DataFrame and round the predictions to binary results
predictions_df = pd.DataFrame(y_test_pred, columns=["predictions"])
predictions_df["predictions"] = round(predictions_df["predictions"], 0)
predictions_df.head()

,predictions
0,0.0
1,0.0
2,0.0
3,1.0
4,0.0


### Step 4: Display a classification report with the y test data and predictions

In [39]:
# Print the classification report with the y test data and predictions
print(classification_report(y_test, predictions_df["predictions"].values))

              precision    recall  f1-score   support

           0       0.46      0.88      0.61       180
           1       0.63      0.17      0.27       220

    accuracy                           0.49       400
   macro avg       0.55      0.53      0.44       400
weighted avg       0.56      0.49      0.42       400



---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.

3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.

**1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.**
You would need data similar to the data included in loans_df with the addition of data including which options each student took and their success with each of those options. A neural network like an RBM could then compare previous students loan option selections and use that information recommended loan options for new users.

**2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.**
I think context-based filtering would be best for this situation. Context-based filtering takes into account additional contextual data, like in this case, the students academic success and credit history. This means, the loan recommendation system wouldn't recommened by what others chose, or similar products, but by how well a users credit and student information align with each loan.

**3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.**
Two challenges within recommending student loans are lack of initial data and data biases. Student often don't have a developed credit history. Any recommendation algorithm would have to predict the loan based off of very little information on the students ability to pay back the loan. Additionally, the data the model is built on could be biased. In the past, if people were providing worse loans to people of specific demographics based on personal biases, those biases would carry over to the model. This is a concern because the model should be designed for maximum fairness its recommendation of loans.
